In [1]:
import re
import pickle
import pandas as pd
from textblob import TextBlob
from spellchecker import SpellChecker
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [2]:
RANDOM_SEED = 13

In [3]:
df = pd.read_excel("sentences/sentences.xlsx")
df.head()

,Intenção,Sentença
0,Consultar saldo da conta,Qual o meu saldo
1,Interagir com a luz ou o ar-condicionado,Ligue a luz
2,Consultar saldo da conta,Me diga meu saldo
3,Consultar saldo da conta,Qual o meu saldo?
4,Interagir com a luz ou o ar-condicionado,está muito claro


## Cleaning the Dataset

In [4]:
df["Sentença"] = df["Sentença"].str.lower()
df["Sentença"] = df["Sentença"].str.replace(r"\s\-\s|\-\-+", " ", regex=True)
df["Sentença"] = df["Sentença"].str.replace(r"[^\w\s\-]", " ", regex=True)
df["Sentença"] = df["Sentença"].str.replace("foxbot ", "", regex=False)
df.head()

,Intenção,Sentença
0,Consultar saldo da conta,qual o meu saldo
1,Interagir com a luz ou o ar-condicionado,ligue a luz
2,Consultar saldo da conta,me diga meu saldo
3,Consultar saldo da conta,qual o meu saldo
4,Interagir com a luz ou o ar-condicionado,está muito claro


## Adapting Data for MultinomialNB

In [5]:
# Adapts data to be used by the MultinomialNB model and already splits into X and y.
v = CountVectorizer()
def adaptToModel(df, objective=None, vectorizer=v):
    txts = df["Sentença"].tolist()
    if objective == "train":
        counts = vectorizer.fit_transform(txts)
        return counts, df["Intenção"]
    elif objective == "test":
        counts = vectorizer.transform(txts)
        return counts, df["Intenção"]
    else:
        raise ValueError("Defina o objetivo ('train' ou 'test')")

## Split Dataset into Train and Test

In [6]:
# Splits dataset into stratified train and test sets.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_index, test_index in split.split(df, df["Intenção"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

## Training the Model

In [7]:
model = MultinomialNB()
X_train, y_train = adaptToModel(strat_train_set, objective="train")
model.fit(X_train, y_train)


MultinomialNB()

## Testing the model

In [8]:
X_test, y_test = adaptToModel(strat_test_set, objective="test")

In [9]:
pred = cross_val_predict(model, X_test, y_test, cv=10, n_jobs=-1)

In [10]:
output = pd.DataFrame()
output["Sentença"] = strat_test_set["Sentença"]
output["Real"] = y_test
output["Predicted"] = pred
output.head()

,Sentença,Real,Predicted
81,quanto dinheiro tenho,Consultar saldo da conta,Consultar saldo da conta
297,vai tá abafado hoje,Obter informações relativas ao clima,Obter informações relativas ao clima
349,aumente a potência do ar-condicionado,Interagir com a luz ou o ar-condicionado,Interagir com a luz ou o ar-condicionado
331,será que amanhã vai sair um sol,Obter informações relativas ao clima,Obter informações relativas ao clima
339,vai dar pra tomar um sol hoje,Obter informações relativas ao clima,Obter informações relativas ao clima


In [11]:
score = accuracy_score(output["Real"], output["Predicted"])
print(f"Acurácia: {score * 100:.2f}%")

Acurácia: 88.04%


## Function to Predict the Class of a New Sentence

It checks if the sentence is in portuguese, corrects and classifies it.

In [12]:
def predNewSentence(txt, predModel=model, vectorizer=v):
    if TextBlob(txt).detect_language() == "pt":
        sentence = ""
        text = re.sub(r"\s\-\s|\-\-+", " ", txt.lower())
        text = re.sub(r"[^\w\s\-]", " ", text)
        text = re.sub(r"foxbot ", "", text)
        spellCheck = SpellChecker(language="pt")
        for word in text.split():
            sentence += spellCheck.correction(word) + " "
        counts = vectorizer.transform([sentence])
        return predModel.predict(counts)[0]
    else:
        raise ValueError("O modelo só é capaz de trabalhar com frases em português!")

## Training the Model with the Complete Dataset

Como o modelo já foi validado podemos treinar o modelo com toda a base de dados para ter previsões ainda melhores.

In [13]:
X, y = adaptToModel(df, objective="train")
model.fit(X, y)

MultinomialNB()

## Predicting a few Examples

In [14]:
predNewSentence("Acende a luz por favor!", predModel=model)

'Interagir com a luz ou o ar-condicionado'

In [15]:
predNewSentence("Foxbot, como tá a minha conta bancária?", predModel=model)

'Consultar saldo da conta'

In [16]:
predNewSentence("Vai fazer sol depois de amanhã?", predModel=model)

'Obter informações relativas ao clima'

In [17]:
predNewSentence("Marque uma consulta com o fisioterapeuta para segunda às 10 horas.", predModel=model)

'Não sei'

## Saving the Model

In [18]:
pickle.dump(model, open("models/model_v0.sav", "wb"))
pickle.dump(v, open("vectorizers/vectorizer_v0.sav", "wb"))